In [1]:
import pandas as pd
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
df = pd.read_csv("/Users/sahin9699/Downloads/spotify_millsongdata.csv")

In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [9]:
df.head(10)

,artist,song,text
0,Dusty Springfield,A Love Like Yours,I broke your heart \r\nAnd I made you blue \...
1,Gloria Estefan,Bad Boy,"Bad boy, bad boy \r\nBad boy, bad boy \r\nBo..."
2,Billie Holiday,My Old Flame,My old flame \r\nI can't even think of his na...
3,One Direction,Better Than Words,Better than words \r\nBut more than a feeling...
4,You Am I,Cathy's Clown,The weightlifter and a library tech you know ...
5,X-Ray Spex,Highly Inflammable,Highly inflammable \r\nWaiting to explode \r...
6,Green Day,Fashion Victim,He's a fashion victim of his own time \r\nIn ...
7,The Beatles,Besame Mucho,"Esame besame mucho, \r\nEach time I bring you..."
8,Pearl Jam,Open Road,Can I have some water up? \r\n \r\n(You're t...
9,Men At Work,Sail To You,Now there is this place \r\nWith so much spac...


In [10]:
df['text'][0]

"I broke your heart  \r\nAnd I made you blue  \r\nInstead of hurting back  \r\nYou said I love you  \r\nThat very day you won my heart  \r\nAnd my love for you shall never part  \r\n  \r\nYou have such an understanding way  \r\nAnd I appreciate you need what I say  \r\n'Cause a love like yours don't come knock, knock, knock, knocking  \r\nKnock, knocking everyday  \r\n  \r\nAlthough I treated you bad from the start  \r\nWithout question you gave me your heart  \r\nNo, a love like yours don't come knock, knock, knock, knocking  \r\nKnock, knocking everyday  \r\n  \r\nIt's the little things you do  \r\nOh, that mean so much  \r\nSo very, very much  \r\n(Consideration), I never knew  \r\nThat is, until I met you  \r\nBelieve me  \r\n  \r\nYou made my life complete  \r\nAnd I know our love with all of my mind  \r\nYou can believe going to hold on tight  \r\n'Cause a love like yours don't come knock, knock, knock, knocking  \r\nKnock, knocking everyday  \r\n  \r\nNo, a love like yours don't

In [11]:
df.shape

(5000, 3)

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.01543869, 0.01546077, ..., 0.00704821, 0.02116827,
       0.00637553])

In [18]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [19]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [24]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))